In [ ]:
!git clone https://github.com/Karandh1r/TextMiningAssignment-1.git

Cloning into 'TextMiningAssignment-1'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 244 (delta 86), reused 217 (delta 59), pack-reused 0
Receiving objects: 100% (244/244), 16.72 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a360506cd999f59d174556374df50ff31495f18327cc2778a5f9ceddb216271f
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=04f5fa3f3d071076553e3bb75e91718369b2b2cf5be69246255becc0691847aa
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import pyLDAvis
import pyLDAvis.sklearn   
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
  

In [ ]:
le = LabelEncoder()
ohe = OneHotEncoder()

file_name_movies = 'TextMiningAssignment-1/moviedata.csv'
file_movie_reviews = 'TextMiningAssignment-1/reviewdata.csv'

df_movie_reviews = pd.read_csv(file_movie_reviews)
print(df_movie_reviews.columns)

Index(['MovieId', 'UserReviews'], dtype='object')


In [ ]:
def lemmatize_words(text):
        lemmatizer = WordNetLemmatizer()
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words]
        return ' '.join(words)

def stemming_words(text):
        ps = PorterStemmer()
        words = text.split()
        words = [ps.stem(word) for word in words]
        return ' '.join(words) 

df_movie_reviews['UserReviews'] = df_movie_reviews['UserReviews'].apply(lemmatize_words)
df_movie_reviews['UserReviews'] = df_movie_reviews['UserReviews'].apply(stemming_words) 

vectorizer = CountVectorizer(stop_words='english',max_features=50)
X = vectorizer.fit_transform(df_movie_reviews['UserReviews'])
column_names = vectorizer.get_feature_names_out()
count_vec_columns = column_names
vectorized_df = pd.DataFrame(X.toarray(),columns=column_names)
num_topics = 7
lda_model_DH = LatentDirichletAllocation(n_components=num_topics, 
                                         max_iter=100, learning_method='online')
LDA_DH_Model = lda_model_DH.fit_transform(vectorized_df)
print("SIZE: ", LDA_DH_Model.shape) 

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic:  ", idx)  
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print_topics(lda_model_DH, vectorizer, 15)
word_topic = np.array(lda_model_DH.components_)
word_topic = word_topic.transpose()
num_top_words = 15
vocab_array = np.asarray(column_names)
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model_DH, X, vectorizer)


SIZE:  (879, 7)
Topic:   0
[('great', 125.68793672870436), ('harley', 111.78571698177707), ('quinn', 86.63064346421179), ('fun', 84.80716620692405), ('action', 81.57538899408307), ('charact', 21.29387059007699), ('stori', 12.254532326503316), ('cast', 10.777223016902516), ('did', 9.353633112829122), ('becau', 8.587357147347207), ('better', 8.437296183643838), ('wa', 5.291198237402256), ('way', 4.991852470153668), ('know', 1.9291308478034144), ('ha', 1.4294947960177502)]
Topic:   1
[('thi', 187.76104484626907), ('just', 136.20420533623442), ('bad', 132.48193246806622), ('good', 101.99575809975839), ('look', 80.4445663676711), ('movi', 74.99618893936075), ('life', 66.00023532696915), ('make', 58.651463258338175), ('like', 55.65907345180743), ('wa', 53.084914528192705), ('way', 52.045378862457405), ('cast', 44.378430649639036), ('realli', 42.44350225601857), ('ha', 40.47165716657543), ('charact', 34.00618640059212)]
Topic:   2
[('film', 424.1752939800073), ('thi', 220.41226105116962), ('l

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.020662  0.070885       1        1  23.428329
5      0.092168 -0.064320       2        1  19.897959
3      0.101133 -0.007288       3        1  15.560279
1      0.100889 -0.106754       4        1  15.254191
6      0.121552 -0.178113       5        1  11.297428
4      0.023841  0.349241       6        1   7.854565
0     -0.460245 -0.063650       7        1   6.707249, topic_info=     Term        Freq       Total Category  logprob  loglift
13   film  535.000000  535.000000  Default  30.0000  30.0000
29   movi  541.000000  541.000000  Default  29.0000  29.0000
44     wa  683.000000  683.000000  Default  28.0000  28.0000
37  scene  157.000000  157.000000  Default  27.0000  27.0000
16  great  160.000000  160.000000  Default  26.0000  26.0000
..    ...         ...         ...      ...      ...      ...
15   good    0.147458  250.673972   Topic7  -8.3045  -4.7364
27   make    0.147332  174.923806   Topic7  -8.3053  -4.3774
26   love    0.147291  162.171584   Topic7  -8.3056  -4.3020
23   like    0.147267  291.937017   Topic7  -8.3058  -4.8901
29   movi    0.147215  541.249749   Topic7  -8.3061  -5.5078

[275 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.547646     act
0         2  0.268235     act
0         3  0.011176     act
0         4  0.044706     act
0         5  0.122941     act
...     ...       ...     ...
47        4  0.449997     way
47        7  0.043269     way
48        6  0.994861   woman
49        5  0.087994  wonder
49        6  0.905082  wonder

[183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 4, 2, 7, 5, 1])